In [ ]:
from datetime import datetime, timedelta
from pydap.cas.urs import setup_session
from dask.diagnostics import ProgressBar
import xarray as xr

In [ ]:
dt0 = datetime(2014, 3, 12, 0)
dt1 = datetime(2019, 1, 1, 0)
done = False
dt = dt0
url, time = [], []
while not done:
    time.append(dt + timedelta(minutes=15))
    url.append(f'https://gpm1.gesdisc.eosdis.nasa.gov/opendap/hyrax/GPM_L3/GPM_3IMERGHHE.05/{dt.year}/{str(dt.timetuple().tm_yday).zfill(3)}/3B-HHR-E.MS.MRG.3IMERG.{dt.year}{str(dt.month).zfill(2)}{str(dt.day).zfill(2)}-S{str(dt.hour).zfill(2)}{str(dt.minute).zfill(2)}00-E{str(dt.hour).zfill(2)}{str(dt.minute+29).zfill(2)}59.{str(dt.hour*60+dt.minute).zfill(4)}.V05B.HDF5')
    if dt == dt1:
        done = True
    dt = dt + timedelta(minutes=30)
session = setup_session('davidbrochart', 'DavidBrochart0', check_url=url[0])

In [ ]:
with ProgressBar():
    ds = xr.open_mfdataset(url, concat_dim='time', parallel=True, engine='pydap', backend_kwargs={'session': session})

In [ ]:
ds = ds.assign_coords(time=time)

In [ ]:
ds.to_zarr('gpm_dataset')